<a href="https://colab.research.google.com/github/balajisriraj/Twitter-Summary-mini-Project/blob/main/Twitter_Trend_Finder_In_Progress_V0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Flow:

- Connect to Twittter API using the required creds
- Find the Top trending Hashtags
- Then filter the hashtags which are in English language
- Filter Top 3 Hashtags from that
- For these 3 hashtags, fetch 100 Tweets individually
- Combain all the 100 tweets into one big corpus
- Do basic Data Cleaning & Processing for better Tokenization
- Using Text to Text transformer t5 model Summarize the tweets
- Show Summarized text output for each Hashtag

Import Python Modules

In [149]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from googletrans import Translator
import GetOldTweets3 as got
import datetime

In [142]:
#Past the keys from your local drive

In [143]:
# Consumer key authentication(consumer_key,consumer_secret can be collected from our twitter developer profile)
auth = OAuthHandler(consumer_key, consumer_secret)
# Access key authentication(access_token,access_token_secret can be collected from our twitter developer profile)
auth.set_access_token(access_token, access_token_secret)
# Set up the API with the authentication handler
api = API(auth)

In [144]:
WOE_ID = 2295424 # Where on Earth id can be extracted from https://nations24.com/world-wide
lan_find = Translator() # for finding teh language of the hashtags

In [145]:
def get_location_trends(locations, auth,n_hashtags,lang):
  api = API(auth)
  trends = api.trends_place(locations)
  data = trends[0]
  trends_data = data['trends']
  global tred_data
  tred_data = []
  for info in trends_data:
    tred_data.append([info['name'],info['tweet_volume'],lan_find.detect(info['name']).lang ] )
  tred_data = pd.DataFrame(tred_data, columns = list(['Hashtag',
                                                    'Tweet_Volume', 'Language'])).sort_values(by = ['Tweet_Volume'],ascending = False)
  #select english language trends
  tred_data = tred_data[tred_data.Language == lang]
  #select top 3 trends
  tred_data = tred_data.nlargest(n_hashtags,columns=['Tweet_Volume'])
  return tred_data

In [146]:
df_trending = get_location_trends(WOE_ID,auth,3,'en')
df_trending

,Hashtag,Tweet_Volume,Language
24,#PCAs,3193948.0,en
25,#SavageLoveRemix,1228436.0,en
11,#TrumpHasCovid,475296.0,en


In [215]:
hashtag_list = list(df_trending['Hashtag'])

In [217]:
for htag in hashtag_list:
  today = datetime.datetime.now().strftime("%Y-%m-%d")
  tweets = tweepy.Cursor(api.search,
              q=htag,
              lang="en",
              since=today).items(5)
  tweets= [tweet.text for tweet in tweets]
  print(htag)
  print('')
  print(tweets)

#PCAs

['@Elser92259846 @BTS_twt Love \n#TheGroup #BTS #PCAs \n@BTS_twt', 'RT @7withArmy: I vote for #BTS under #TheGroup category at #PCAs 2020 \n@BTS_twt', 'RT @perfectlyswift2: I voted for #OnlyTheYoung for #TheSoundtrackSong  #PCAs \n@taylorswift13', 'RT @mygtrivias: 2,000 retweets challenge! Keep your eyes on the goals.\n\nI vote for #Dynamite for #TheMusicVideo category at #PCAs 2020 @BTS…', 'I vote for #BTS under #TheGroup category at #PCAs 2020 \n@BTS_twt  the truth untold']
#SavageLoveRemix

["RT @Metro_Ents: Jungkook takes the lead on BTS's #SavageLoveRemix and our golden maknae is all grown up https://t.co/LM1b7fTxHp", 'RT @SugaChartData: #SavageLoveRemix has entered the Top 10 on US iTunes at #6', 'RT @bts_bighit: #BTS #방탄소년단 #SavageLoveRemix OUT NOW\n \n🎶 https://t.co/r7grqx9OLM\n🎶 https://t.co/dS6OAheQBT \n\n@Jawsh_685 x @jasonderulo x #B…', 'RT @kkoobap: ONLY JUNGKOOK CAN MAKE SWEARING SOUND SO CUTE AND SEXC AT THE SAME TIME\n\n#SavageLoveRemix @BTS_twt\nhttps://t.co/A2E

In [201]:
tweets

['RT @CEDRdigital: Images of #GlassFire burning in #NapaCounty and #SonomaCounty. One of over 2 dozen major #wildfires in #California',
 'Man made #ClimateChange does not only effect us.  What gives us the right to sit back while this stuff happens?… https://t.co/ovM7j13Gn5',
 'RT @CEDRdigital: Images of #GlassFire burning in #NapaCounty and #SonomaCounty. One of over 2 dozen major #wildfires in #California',
 'Images of #GlassFire burning in #NapaCounty and #SonomaCounty. One of over 2 dozen major #wildfires in #California https://t.co/LynaaLcohG',
 'RT @RedCrossNorCal: Don’t let your guard down! If you are in an at-risk area for #wildfires, be prepared to evacuate. The weather remains h…']

'2020-10-02'